In [1]:
import pandas as pd
import numpy as np
import jieba

In [2]:
list1 = jieba.lcut("在非洲，每60秒就過了一分鐘", cut_all=False)
list1

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\A3029\AppData\Local\Temp\jieba.cache
Loading model cost 4.088 seconds.
Prefix dict has been built succesfully.


['在', '非洲', '，', '每', '60', '秒', '就', '過了', '一分鐘']

In [3]:
df = pd.read_excel(r'201704全產品對話記錄_time.xlsx')

In [4]:
df.head()

,TenantId,ChatId,ICRId,Skill,CreateTime,DisposeTime,QueueMS,Duration,AgentId,MessageTime,SenderId,SenderName,MessageType,Link,Content
0,TW,00033383f8c6453a9cffcd9e5d3eb7e8,b4a47416-c9d1-0348-e005-ffffff15b940,PAD,2017-04-22 12:51:46.280,2017-04-22 12:55:46.197,107,128,A0223,2017-04-22 12:53:55.357,A0223,Huangshin,text,NaN,歡迎使用華碩即時訊息服務，請問有甚麼地方可以協助您?
1,TW,00033383f8c6453a9cffcd9e5d3eb7e8,b4a47416-c9d1-0348-e005-ffffff15b940,PAD,2017-04-22 12:51:46.280,2017-04-22 12:55:46.197,107,128,A0223,2017-04-22 12:55:42.967,A0223,Huangshin,text,NaN,[系統公告]您好! 您已超過2分鐘沒有回應，請問您是否有需要我服務的地方?
2,TW,0006e706ea984def8a820c22836aaad6,b4a47466-d670-0550-c319-ffffff15b3d2,NB,2017-04-05 16:01:30.440,2017-04-05 16:10:04.063,0,481,A0233,2017-04-05 16:01:45.497,A0233,Andy,text,NaN,歡迎使用華碩即時訊息服務，請問有甚麼地方可以協助您?
3,TW,0006e706ea984def8a820c22836aaad6,b4a47466-d670-0550-c319-ffffff15b3d2,NB,2017-04-05 16:01:30.440,2017-04-05 16:10:04.063,0,481,A0233,2017-04-05 16:02:37.537,aliceli9990@gmail.com,Alice Li,text,0006e706ea984def8a820c22836aaad6C0001,我的t102h一直重開機不能用
4,TW,0006e706ea984def8a820c22836aaad6,b4a47466-d670-0550-c319-ffffff15b3d2,NB,2017-04-05 16:01:30.440,2017-04-05 16:10:04.063,0,481,A0233,2017-04-05 16:03:03.950,aliceli9990@gmail.com,Alice Li,text,0006e706ea984def8a820c22836aaad6C0002,平板電腦説找筆電處理


In [5]:
df['Content'] = df['Content'].str.upper()

In [6]:
from cleanse import separateUserContent,separateAllContent
from random import shuffle, randint
from gensim.models import word2vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt

D:\Anthony\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
all_content, all_link = separateAllContent(df)

In [8]:
#對話形式
all_content[:2]

[['我的T102H一直重開機不能用',
  '平板電腦説找筆電處理',
  '電源有重新插拔過嗎',
  'IMG_0264.JPG',
  'IMG_0265.JPG',
  '有等他跑完過嗎',
  '像平板那款已跑10分鐘了',
  '一直說處理更新',
  '目前可能是在更新 建議您先等他跑完',
  '我第一次使用這款這常會發生嗎？',
  '若是第一次使用',
  '一定會跑更新',
  '剛買一個月用二三次而已',
  '最近微軟常在更新 這樣其實算是正常的 您先等他跑完看看',
  '之前不是已經跑過',
  '可能微軟又有發布更新了',
  '好吧！大約會多久',
  '不一定 這部分無法跟您確認準確的時間',
  '好的謝謝你',
  '再等等好了',
  '好的不會',
  '請問還有什麼地方可以為您服務的?'],
 ['你好',
  '請問手機 RAM 保持多少 比較好呢？',
  '這方面需視後續需使用的應用程式判斷，一般來說也建議您可隨時將無須使用的應用程式關閉',
  '如RAM 不足系統會自動將部分應用程式自動關閉',
  'RAM 只要有看APP 或網頁都會消耗嗎？',
  '視的',
  '只要手機 不會變慢 消耗記憶體都是正常的?',
  '一般來說只要有應用程式執行均會站據記憶體空間使用']]

In [9]:
#產生把 CONTENT 跟 LINK 合在一起的set
def generateAllData(all_content, all_link): 
    all_set = []
    

    for i in range(len(all_content)):
        tmp_list = []
       
        for j in range(len(all_content[i])):
            tup = (all_content[i][j], all_link[i][j])
            tmp_list.append(tup)
        
        all_set.append(tmp_list)
        
    return all_set

In [10]:
all_set = generateAllData(all_content, all_link)
shuffle_set = all_set
#shuffle(shuffle_set) #先不要SHUFFLE 

In [11]:
shuffle_set[:2]

[[('我的T102H一直重開機不能用', 'c'),
  ('平板電腦説找筆電處理', 'c'),
  ('電源有重新插拔過嗎', 's'),
  ('IMG_0264.JPG', 'c'),
  ('IMG_0265.JPG', 'c'),
  ('有等他跑完過嗎', 's'),
  ('像平板那款已跑10分鐘了', 'c'),
  ('一直說處理更新', 'c'),
  ('目前可能是在更新 建議您先等他跑完', 's'),
  ('我第一次使用這款這常會發生嗎？', 'c'),
  ('若是第一次使用', 's'),
  ('一定會跑更新', 's'),
  ('剛買一個月用二三次而已', 'c'),
  ('最近微軟常在更新 這樣其實算是正常的 您先等他跑完看看', 's'),
  ('之前不是已經跑過', 'c'),
  ('可能微軟又有發布更新了', 's'),
  ('好吧！大約會多久', 'c'),
  ('不一定 這部分無法跟您確認準確的時間', 's'),
  ('好的謝謝你', 'c'),
  ('再等等好了', 'c'),
  ('好的不會', 's'),
  ('請問還有什麼地方可以為您服務的?', 's')],
 [('你好', 'c'),
  ('請問手機 RAM 保持多少 比較好呢？', 'c'),
  ('這方面需視後續需使用的應用程式判斷，一般來說也建議您可隨時將無須使用的應用程式關閉', 's'),
  ('如RAM 不足系統會自動將部分應用程式自動關閉', 's'),
  ('RAM 只要有看APP 或網頁都會消耗嗎？', 'c'),
  ('視的', 's'),
  ('只要手機 不會變慢 消耗記憶體都是正常的?', 'c'),
  ('一般來說只要有應用程式執行均會站據記憶體空間使用', 's')]]

In [12]:
def generateDialogueTupleList():
	tmp_c = ""
	tmp_s = ""
	all_tup_dia = []


	for small_list in shuffle_set:
	    tmp_c = ""
	    tmp_s = ""
	    previous_status = ""
	    all_tup_list = []
	    for tup in small_list:
	        if previous_status == "":
	            if tup[1] == 's':
	                continue
	            else:
	                tmp_c = str(tmp_c) + str(tup[0])
	                previous_status = 'c'
	        elif previous_status == 's':
	            # s -> s
	            if tup[1] == 's':
	                tmp_s = str(tmp_s) + " " + str(tup[0])
	            # s -> c
	            else:
	                all_tup_list.append((str(tmp_c), str(tmp_s)))
	                tmp_c = str(tup[0])
	                tmp_s = ""
	                previous_status = 'c'
	        else:
	            # c -> s
	            if tup[1] == 's':
	                tmp_s = str(tmp_s) + str(tup[0])
	                previous_status = 's'
	            # c -> c
	            else:
	                tmp_c = str(tmp_c) + " " + str(tup[0])
	    all_tup_list.append((str(tmp_c), str(tmp_s)))
	    all_tup_dia.append(all_tup_list)
	return all_tup_dia

In [13]:
#對話裡面是QA TUPLE形式
all_list = generateDialogueTupleList()

In [14]:
all_list[:2]

[[('我的T102H一直重開機不能用 平板電腦説找筆電處理', '電源有重新插拔過嗎'),
  ('IMG_0264.JPG IMG_0265.JPG', '有等他跑完過嗎'),
  ('像平板那款已跑10分鐘了 一直說處理更新', '目前可能是在更新 建議您先等他跑完'),
  ('我第一次使用這款這常會發生嗎？', '若是第一次使用 一定會跑更新'),
  ('剛買一個月用二三次而已', '最近微軟常在更新 這樣其實算是正常的 您先等他跑完看看'),
  ('之前不是已經跑過', '可能微軟又有發布更新了'),
  ('好吧！大約會多久', '不一定 這部分無法跟您確認準確的時間'),
  ('好的謝謝你 再等等好了', '好的不會 請問還有什麼地方可以為您服務的?')],
 [('你好 請問手機 RAM 保持多少 比較好呢？',
   '這方面需視後續需使用的應用程式判斷，一般來說也建議您可隨時將無須使用的應用程式關閉 如RAM 不足系統會自動將部分應用程式自動關閉'),
  ('RAM 只要有看APP 或網頁都會消耗嗎？', '視的'),
  ('只要手機 不會變慢 消耗記憶體都是正常的?', '一般來說只要有應用程式執行均會站據記憶體空間使用')]]

In [15]:
real_qa =[]
count = 0
for dia in all_list:#找出一個對話李真正的問題與真正的答案
    real_q = ""
    real_a = ""
    count += 1
    for num in range(len(dia)):
        real_q = real_q + dia[num][0] #客人第一句畫室問題 如果客服問問題 繼續CONCAT客人的問題
        
        if "請問還有什麼地方可以為您服務" in dia[num][1] and dia[num][1][0] == "請":#如果第一個字不是請代表有可能答案混在裡面
            if num > 0 :
                num -= 1
                real_a = dia[num][1]
                
                while(len(dia[num][1]) < 3) :#如果前一句還是小於三個字就在找上一句
                    if num > 0:
                        num -= 1
                        real_a = dia[num][1]
                    else:
                        break
                
                real_qa.append((real_q,real_a))
            break
        elif  "為確保服務品質，可否請您針對此次的服務於畫面上方做滿意度評價"in dia[num][1]:
            if num > 0 :
                num -= 1
                real_a = dia[num][1]
                while(len(dia[num][1]) < 3) :#如果前一句還是小於三個字就在找上一句
                    if num > 0:
                        num -= 1
                        real_a = dia[num][1]
                    else:
                        break
                
                real_qa.append((real_q,real_a))
            break
        
        
        if "請問" in dia[num][1] or "嗎" in dia[num][1] or "?" in dia[num][1] or "呢" in dia[num][1] :
             #克服通常會詢問一些問題後給出真正答案 所以這些問題省略 不放進真正答案
            continue
        elif "不好意思，請您稍候片刻，我會立即為您查詢，謝謝" in dia[num][1]:
            continue
        elif "由於目前所有客服人員忙線中" in dia[num][1]:
            continue
        elif len(dia[num][1]) < 4:#克服回太短的話通常都不是答案
            continue
        elif "您稍後一下幫您確認" in dia[num][1]:
            continue
        
       
        
        real_a = real_a + dia[num][1]
        break
    
    

    if len(real_a)  > 3 and len(real_q) > 3: #真正的問題與答案如果其中意個是空白的就不加入
        real_qa.append((real_q,real_a))
    
print(len(real_qa))

9327


In [16]:
real_qa[:2]

[('我的T102H一直重開機不能用 平板電腦説找筆電處理IMG_0264.JPG IMG_0265.JPG像平板那款已跑10分鐘了 一直說處理更新',
  '目前可能是在更新 建議您先等他跑完'),
 ('你好 請問手機 RAM 保持多少 比較好呢？',
  '這方面需視後續需使用的應用程式判斷，一般來說也建議您可隨時將無須使用的應用程式關閉 如RAM 不足系統會自動將部分應用程式自動關閉')]

In [17]:
real_qa[8500]

('為什麼我的筆電連WIFI都會斷掉 出現三角形警示標誌 可是分享器是沒有問題的 已經好幾次這樣了 別的裝置也都正常 唯獨你們家的筆電這樣',
 '請提供您的機台序號幫您查詢，序號在機台底部寫在一張條碼貼紙上，第三碼是英文N那一串')

In [18]:
def generateTVData(shuffle_set): #產生train validation set 以1:10的比率 
    training_set = []
    validation_data_set = []

    for i in range(len(shuffle_set)):
        tmp_list = []
        if i < len(shuffle_set) / 10:
            validation_data_set.append(shuffle_set[i])
        else:
            training_set.append(shuffle_set[i])

    return training_set, validation_data_set


In [19]:
training_tup_list, validation_tup_list = generateTVData(real_qa)

In [20]:
len(validation_tup_list)

933

In [21]:
test_tup_df = pd.DataFrame.from_records(validation_tup_list)#存起來
test_tup_df.to_pickle("validation_tup_list.pkl")

In [22]:
test_tup_df = pd.DataFrame.from_records(training_tup_list)#存起來
test_tup_df.to_pickle("training_tup_list.pkl")

In [23]:
#網路上找的停用詞，把不重要的詞刪掉 對WORD2VEC訓練應該比較有幫助
with open("stopwords","r", encoding='utf-8') as f:
    content = f.read()
    stop_list = content.split('\n')

In [24]:
list3 = []#把TRAIN 切詞  主要是為了丟進WORD2VEC
for i in range(len(training_tup_list)):
    
    list1 = jieba.lcut(training_tup_list[i][0], cut_all = False)
    list2 = jieba.lcut(training_tup_list[i][1], cut_all = False)
    
    ques_list = []
    ans_list = []
    
    for i in list1:
        if i not in stop_list:#停用詞不用加入訓練
            ques_list.append(i)
    for i in list2:
        if i not in stop_list:
            ans_list.append(i)

    list3.append([ques_list])
    list3.append([ans_list])
list3_df = pd.DataFrame(list3,columns = ['text']) 

In [25]:
corpus = list3_df.text.sample(frac = 1)#洗牌 為了讓WORD2VEC訓練比較好

In [26]:
corpus[:5]

15569    [保固, 發票, 日期, 發票, 機器, 序號, 判定, 保固, 無關, 保固, 註冊, 無...
6015     [不好意思, 平板, 客服, 詢問, 筆記型, 電腦, WINDOWS, 系統, 變型, 筆...
1002     [設群, 軟體, 即時, 收訊, 最近, 延遲, 社群, 軟體, LINE, 臉書, 改善,...
15614                    [找, 設定, 鍵, 桌面, 設定, 鍵, 設定, 時間, 網路]
6732               [您好, 想, 請教, 內部, 空間, 不足, SD, 空間, 儲存, 解決]
Name: text, dtype: object

In [27]:
vec_model = word2vec.Word2Vec(corpus, sg=1, iter=4, size=250)#丟進word2vec
vec_model.save("word2vec.model")
#vec_model = word2vec.Word2Vec.load("word2vec.model")

In [28]:
len(vec_model.wv.vocab)#辭典裡面詞的數量

3609

In [29]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [30]:
print(most_similar(vec_model, ['抱歉', '檢測', '維修', '安全', '網路', '等一下','使用', '5.5', 'GB', '桌上型']))

D:\Anthony\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


   抱歉       cos   檢測       cos  維修       cos   安全       cos    網路       cos  \
0   聲  0.943102   收回  0.935854  取件  0.847228   模式  0.848955  WIFI  0.845612   
1  即時  0.910624   交由  0.932838  費用  0.809531   引導  0.767566    連線  0.839873   
2   通  0.873135  維修師  0.915832  寄出  0.799647  ESC  0.746834    上網  0.823619   
3  非本  0.833689   實機  0.915326  檢修  0.796805   為止  0.743876   基地台  0.822392   
4  註記  0.810297   送測  0.913971  過保  0.796264   請重  0.742520    乙太  0.821412   
5  這裡  0.804718   備註  0.905974  單號  0.786773    字  0.727264    有線  0.820992   
6  尚未  0.798966  工程師  0.905938  寄回  0.786381   拉式  0.724719    無線  0.812114   
7  故無  0.798282  進一步  0.904708   寄  0.784798  左下角  0.723937    共用  0.808850   
8  業務  0.796837   工程  0.904615  安排  0.781393   月亮  0.723380    業者  0.795026   
9   說  0.796747   損壞  0.898184  價格  0.778006   起動  0.722917    分享  0.786558   

  等一下       cos  使用       cos      5.5       cos       GB       cos      桌上型  \
0   稍  0.933144  環境  0.727864      MAX  0.973728  

In [31]:
train_q = []#看看問題跟答案的詞的長度大約是多長
train_a = []
avg_q = 0
avg_a = 0
for i in range(len(training_tup_list)):
    tmp_q = jieba.cut(training_tup_list[i][0], cut_all = False)
    tmp_a = jieba.cut(training_tup_list[i][1], cut_all = False)
    train_q.append(len(list(tmp_q)))
    train_a.append(len(list(tmp_a)))
    avg_q += train_q[i]
    avg_a += train_a[i]
print(avg_q /len(training_tup_list))
print(avg_a /len(training_tup_list))

31.752084822492257
26.767929473433405


In [32]:
print(training_tup_list[6][0])#確定切的詞的數量有對
tmp_q = jieba.cut(training_tup_list[6][0], cut_all = False)
print(list(tmp_q))
print(train_q[6])

維修編號:TWA3H40426 可能有開機密碼:SHAN0803 請告訴維修人員，感恩。張漢文 板橋皇家
['維修', '編號', ':', 'TWA3H40426', ' ', '可能', '有', '開機', '密碼', ':', 'SHAN0803', ' ', '請', '告訴', '維修', '人員', '，', '感恩', '。', '張漢文', ' ', '板橋', '皇家']
23


In [33]:
word_index = {}    #把WORD2VEC DIC 裡面的詞編號 

count = 0
for key in vec_model.wv.vocab.keys():
        word_index[key] = count
        count += 1
        
import pickle

f = open("word_index_dic.pkl","wb")
pickle.dump(word_index,f)
f.close()

In [34]:
ans_word_list = []
for i in range(len(training_tup_list)):#把每個ANS裡面如果有出現詞那個詞就設成1
    count = 0
    list2 = jieba.cut(training_tup_list[i][1], cut_all = False)
    tmp = np.zeros((len(vec_model.wv.vocab), 1))#每一個詞都是一個所有詞的數量的ONEHOT
    for word in list(list2):
        if word in vec_model.wv.vocab and tmp[word_index[word]] == 0:
            tmp[word_index[word]] = 1
            count += 1
    ans_word_list.append((tmp, count))#有把一個句子詞的數量加進裡面

In [35]:
#ans_word_list[:2]

In [36]:
"""
for i in range(len(training_tup_list)):
    if("HTTP" in training_tup_list[i][1]):
        print(training_tup_list[i][1])
"""

'\nfor i in range(len(training_tup_list)):\n    if("HTTP" in training_tup_list[i][1]):\n        print(training_tup_list[i][1])\n'

In [37]:
ans_relevant_set = []

ans_index_set = []
for i in range(len(ans_word_list)):#為了不讓句子在重複的分類李 也就是句子只能出現一次
    ans_index_set.append(i)
    
web_set = []
for i in range(len(training_tup_list)):
    if("HTTP" in training_tup_list[i][1]):#先對網址做分類
        #count+=1
        for j in range(len(training_tup_list[i][1])):
            if j + 2 < len(training_tup_list[i][1]):
                if training_tup_list[i][1][j] == 'H' and training_tup_list[i][1][j+1] =="T"and training_tup_list[i][1][j+2] =="T":#先找網址的頭
                    loc_start = j
                    break
        
        for j in range(loc_start, len(training_tup_list[i][1])):
            loc_end = j
            if training_tup_list[i][1][j].isalpha() or training_tup_list[i][1][j].isdigit() or training_tup_list[i][1][j] in '?=+_/:.-%':#找網址的尾巴 都不是這些字就代表尾巴
                 if j == len(training_tup_list[i][1]) -1:
                        loc_end += 1
            else:
                break
        web_set.append((training_tup_list[i][1][loc_start:loc_end],i))
            
#print(web_set)  
web_index_set =[]
for i in range(len(web_set)):#為了讓網址不要在群裡重複
    web_index_set.append(i)
#print(web_index_set)

for i in range(len(web_set)):
    
    if i  in web_index_set:#沒在裡面代表已經被分過了

        tmp = web_set[i][0].split('/')
        sentence1 = "".join(tmp)
        tmp_list = [web_set[i][1]]
        
        for j in range(i+1,len(web_set)):
            if j  in web_index_set:
                tmp = web_set[j][0].split('/')
                sentence2 = "".join(tmp)

                if(sentence1 == sentence2):
                    tmp_list.append(web_set[j][1])
                    web_index_set.remove(j)
                    ans_index_set.remove(web_set[j][1])#之後的分類也不能再分了
        if len(tmp_list) > 1:#有超過兩個才算一群
            ans_index_set.remove(web_set[i][1])
            ans_relevant_set.append(tmp_list)
print(ans_relevant_set)



'''
with open("ans_classification_set_web.txt","w") as f:
    for i in range(len(ans_relevant_set)):
        f.write("Answer group %d : length is %d\n"% (i, len(ans_relevant_set[i])))
        count = 0
        for j in ans_relevant_set[i]:
            
            f.write("%d:          %s    @@%s@@ \n"% (count,training_tup_list[j[0]][1],j[1]))
            print(j[1])
            count+=1
        
        f.write("\n\n\n")
        f.write("=======================================================\n\n\n")
        '''
alone = 0
alone_set =[]
for num in range(len(ans_word_list)):
    
    
    if num in ans_index_set: #如果沒有在裡面 代表出現過了 就不用找了
        
        threhold = 9 #代表要超過幾個字才算
        
        if ans_word_list[num][1] > threhold:#詞的數量超過門檻也就是比較長的
            tup_list = []
            for j in range(num +1, len(ans_word_list)): #算出這一句跟之後的每一句有幾個詞一樣
                grade = np.sum(ans_word_list[num][0] * ans_word_list[j][0])
                tup_list.append((j,grade))#J是第幾句的意思

            tup_list.sort(key=lambda tup: tup[1], reverse = True)#由大排到小


            if tup_list[0][1] > threhold:#跟這句詞相同分數第一明有10個詞一樣才繼續
                tmp_list = []
                tmp_list.append(num)
                for tup in tup_list:
                    if tup[1] > threhold and tup[0] in ans_index_set:
                        tmp_list.append(tup[0])
                        ans_index_set.remove(tup[0])#出現過了就不能再出現了
                    elif tup[1] <= threhold:
                        break
                if(len(tmp_list) > 1):#有加入東西才算一個新的分類
                    ans_relevant_set.append(tmp_list)
                else:
                    alone +=1
                    alone_set.append(num)
            else:
                alone += 1
                alone_set.append(num)
       
        elif ans_word_list[num][1] > 3:  #詞的數量小於門檻直
            
            tup_list = []
            for j in range(num +1, len(ans_word_list)): #算出這一句跟之後的每一句有幾個詞一樣
                
                if ans_word_list[num][1] <= threhold: #比較的詞數量也必須小於門檻
                    grade = np.sum(ans_word_list[num][0] * ans_word_list[j][0])
                    tup_list.append((j,grade))#J是第幾句的意思

            tup_list.sort(key=lambda tup: tup[1], reverse = True)#猶大排到小
            
            tmp_list = []
            tmp_list.append(num)
            for tup in tup_list:
                if tup[1] == ans_word_list[num][1]  and tup[0] in ans_index_set:#對短的句子詞要全隊才算同類
                    tmp_list.append(tup[0])
                    ans_index_set.remove(tup[0])#出現過了就不能再出獻了
                elif tup[1] < ans_word_list[num][1]:
                        break
            if(len(tmp_list) > 1):#有加入東西才算一個新的分類
                ans_relevant_set.append(tmp_list)
            else:
                alone +=1
                alone_set.append(num)
                
       
    if num % 100 == 0: #為了計算時間
        print(num)

        
            

[[40, 1704, 1916, 1933, 5864, 7568], [72, 1329, 2938, 3900, 6670, 6891, 7622], [113, 7597], [115, 1486, 1993, 2105, 2291, 2719, 6112, 6430, 7407, 7536, 8043, 8365], [116, 896, 6096, 6762, 7865], [129, 133, 590, 2593, 2876, 6630, 7437, 8171, 8188], [137, 1769, 1811, 5191], [218, 370, 372, 404, 418, 542, 1015, 1063, 1076, 1195, 1569, 1690, 1867, 1968, 2020, 2101, 2317, 2989, 3132, 3247, 3302, 3357, 3524, 3615, 3709, 3894, 4303, 4572, 4650, 4859, 5183, 5383, 5865, 5880, 6360, 6399, 6444, 6675, 6785, 6981, 7132, 7183, 7240, 7375, 7574, 7823, 7915, 8192, 8200, 8311, 8373], [366, 1287, 1650, 3315, 3617, 3904, 4961, 5324, 5459, 5665, 7745, 8226], [371, 748, 1272, 4387, 6448, 6990, 7986], [411, 4054, 5252, 6271, 7481], [474, 7818], [500, 1374, 6325], [510, 6328, 6506], [559, 7000], [688, 689], [745, 1257, 1869, 1910, 2466, 2524, 2539, 2635, 2906, 3934, 7236, 7743, 7891], [856, 6635, 7141, 8308], [948, 4779], [1024, 6082], [1049, 3429, 4628, 5340, 6022], [1061, 1834, 3236, 5317, 6914], [1082, 1

In [38]:
print(alone)#沒被分到群的

3798


In [39]:
for i in alone_set:
    print(training_tup_list[i][1])

不好意思，目前只由宋修由工程師檢測維修
您幫我進入安全模式嘗試看看是否還是會有相同的情形產生
設定>應用程式>時鐘>停用後再重新啟用試試看
其他手機也會有支援問題，手邊測試他牌手機是可以，或是您可以確認權限的部分有無開啟
確認VM60-17U5R0A機種是台灣出貨的產品，台灣出貨證明建議您可以洽詢經銷商確認
我這邊是無隨意調閱會員資料的，需由您這邊提供給我 您幫我確認看看有沒有SN或是TW開頭的維修單號
這部分需將傳輸線連接至平板 使用手動方式移動
PLAY商店>左上角三條線>我的應用程式與遊戲>請您看看APP是否需要更新(YOUTUBE)
6.0之後是無法將應用程式安裝在SD卡上的非常抱歉
沒關係的，這邊都會有您的文字紀錄 未命名.JPG
基本上我們針對額外加裝的裝置並沒有測試過 無法保證相容性 不過您可以更新音效驅動看看
若還原後GOOGLE帳戶無法登入  建議將機台送修由人員協助
耳機的接頭，有分為3環4節跟2環3節
您好  再升級系統  會建議您將資料備份後再做操作。
非常抱歉受於此方面非ASUS 相關評測軟體，較無法對於此方面測試 會建議您參考網路用戶使用心得分想了解
這邊比較傾向於售後的技術諮詢 或是 您提供型號幫您確認
這邊會建議您將您的向片或是音樂可以放入SD卡，或是刪除比較少用的程式做緩解
請您幫我檢查 FN+F2   是否有點到飛航。
這邊建議您先接上有線網路 更新無線網卡驅動
那建議您資料備份，先找原通訊行詢問有無相關處理方式，如果由我們這邊是 一年內保固保修 看您的需求
請您稍後幫您確認 這邊確認，上位專員已經協助反應了
最慢5天不包含假日
建議您將系統更新至最新試試看 若還是同情形 需將重要資料備份 系統還原
那可能是驅動還沒安裝 建議您先下載更新
感謝您的建議 比較不好意思 目前華碩尚未釋出新增功能的相關資訊
這邊有確認您的問題點，建議您可以到皇家協助檢測看看，相關問題的部分
檔案管理員>>影像>>照片>>按住不放>>複製至>>SD卡
我們官網註冊部分幫我重新註冊
若系統還原後仍無改善，建議將機台送檢測由工程師協助處理
不好意思，這邊為您查詢沒有降版版本
若風扇 聲音有異常，  建議送檢測，   提醒您備份資料
您必須自備記憶體 再帶到服務點加裝 基本會收500元安裝費 大部份是跟顯卡比較有關係喔
建議您可以到各大3C賣場洽詢經銷商此機種

UP TO 256GB SSD
麻煩您幫我們長壓電源按鈕 直到螢幕熄滅
可以送至維修中心 請人員協助確認看看
已經最新版本，還沒有支援6.0
了解，如果說還是沒出現，請您幫我回到  設定  -> 應用程式  找尋剛剛的 通話 點選   強制停止    再把她重新啟用
若您在保內非人損都是屬於免費
沒有辦法，電話部分建議您撥打0800093456 電話客服
右下角音效符號
會在機台底部或保固卡上，會是英數混合12~15碼，第三碼是字母N
皇家俱樂部營業時間１１點開門
可透過 FN+F9 關閉
這部分建議您嘗試移除新安裝
這邊可協助您操作 工程模式強制打開 不過資料會清除喔
後續部分有發佈 版本 WW-21.40.0.2081  建議您可先更新看看 可操作，系統更新當中的檢查更新選項
在麻煩您提供當初流下的全名 聯繫電話 收件地址 及 TW開頭維修單號。
因為這部分是您的手機認體與軟體異常 需請您備份手機資料後還原
這部分需要手動更新至安卓6.0 因為安卓6.0有些部分程式已移除 和應用程式無法存取SD卡 所以不強迫進行更新
若有硬體異常 會再建議您送修檢測
好的 了解 您可以先於設定 關於 系統更新 來確認是否還有更新版本
中壢 桃園縣中壢市中正路389號3樓320室 中壢皇家俱樂部
您可以於相機 找到齒輪圖案 點選相機恢復原廠設定 在使用看看
建議您到微軟官網 點選 忘記密碼 重設即可
已購買發票上的時間為主
您可以至瀏覽器的設定 網站設定 將翻譯關閉在使用看看
確認到TYPE-C僅傳輸資料
如需螢幕輸出 需使用HDMI轉接
您好，電腦版與手機版的CHROME本身是無法同步的非常抱歉
設定>應用程式>已停用 重新啟用即可
若資料移除後，是無法復原的。
您好 正常網路下載及安裝 並不會有2天的時間 會建議您將產品關機後再開機 能否正常開機使用。
將手機重開機，即可回覆一般模式
因這個部分主要是瀏覽的網頁有無提供縮放功能 建議您可使用CHROME瀏覽器 有內建強制放大功能
確認到目前資訊 沒有辦法喔
您依相同路徑  會員中心>>我的產品>>點選要刪除產品  右邊會有一個刪除的按鈕
型號可以在您機台背面貼紙上
這部分可能是您的皮套有干擾到訊號的部分 建議您使用原廠皮套
建議您先透過機台內部錄音程式測試看看是否是麥克風異常
ZENFONE SELFIE 安卓5.0.2 WW

設定>>應用程式>>LINE>>權限>>麥克風(開啟)
產品硬體規格是固定無法做更動 若再SIM2需透過2G訊號做使用，電信商停止該2G功能，那麼會有無法讀取的狀況喔。
目前請您等待系統運作確認是否可進桌面
請您提供序號 幫您查詢是否還有時段
協助您測試請稍後
不好意思，此區僅服務華碩手機相關產品資訊。
是的 如果還是無法 就需由工程師成灌手機系統 做刪除
不好意思，此部份屬於安卓6.0的一個資安機制，系統還原後會請您輸入原先的GOOGLE帳號才能啟動，如您忘記原先帳號及密碼，只能協助您送修系統重裝
了解，請您先幫我到 安全模式 測試 WIFI 功能 。
控制台>電源>電力提醒 幫我確認看看
這個狀況發生多久了 哪個部分已停止
如果是有相關問題會建議由工程單位這邊幫您檢測後確認，不過的確嗜異您可以做更換電池的動作嘗試
有裂痕的部分會建議您可以讓我們工程單位可以協助檢修
提醒您更新至6.0版本 APP不支援移至SD卡功能
幫您查詢S/N序號正常
好的 後續這邊會在請人員確認後盡速與您回電說明
機台的記憶體是焊在主機板上的，無法作擴充更換的動作
內建部分無會圖軟體 無繪圖軟體
抱歉，  是想確認 顯示晶片 的什麼規格  尚未購入 是不會有機器序號
您的數據網路是SIM2為主要卡槽 故 2槽是到4G的 另一曹是3G
這方面的圖片是硬體資訊，  版本的部分是在軟體資訊當中 
左下 WINDOWS 齒輪設定
基本款的的ZENFONE3(ZE552KL和ZE520KL)  ZF3 ULTRA影音版螢幕較大、再來旗艦機ZF3 DELUXE 他的硬體規格會比較好、ZF3 MAX電池容量大，硬體上的規格沒有比較好，ZF3 ZOOM相機規格比較好 收訊不良部分是尚未有這方面的資料回報
有可能是軟體聯線異常，若有出現此狀況建議直接用流覽器到華碩官網登入會員註冊
您可以在華碩客服>右下角人頭>問答紀錄  看到今天的對話內容
麻煩您先登入華碩會員帳號，於華碩數位學園課程中取消該場次報名
建議您嘗試更換其他的鍵盤使用
右上角三個點 鍵盤設定>>鍵盤主題
目前的確還未放置官網
請您到相機設定>恢復預設值，遊戲請您重新安裝
明天有5個時段  12點 下午2點~5點  一個小時一個時段
於6.0後目前已無提供音樂APP功能。
這部分建議您嘗試做WINDOWS 更新
目前最新應是ZE553KL ZEN

建議先測試其他SIM卡做測試看看
不好意思，維修報價都是由工程師說明的，您可以至皇家俱樂部 或者 客服中心 協助收件  請工程師檢測維修  。
ZB551KL 幫我們到設定>>應用程式>>圖片庫>>清除資料
好的 這邊可以協助您預約現場服務中心
這邊維修端尚未記錄寄出時間，這邊無法直接幫您確認不好意思
了解，這部分您可以將機台帶往皇家俱樂部，或透過客服中心 由 工程師檢測判斷比較準確。
請您先確認您ZF3手機 設定>是否有開發人員選項
這部分情況的確是比較特殊，包誇如果只有裝上一張SIM卡會無法有訊號的部分，因為您是剛購買，會建議您可以先到原購買處，請現場人員協助確認，是否有相關的問題和後續的服務
手機的ASUS雲端是叫 WEBSTORAGE
沒關係，若您有這部分的疑慮，建議在您方便的時間將機台帶到我們全省皇家幫您實際查看確認
請您到設定>顯示>字型大小試試看
這邊會建議您洽詢英才皇家做洽詢
這方面會建議您透過。智能管家>權限管理>自啟動管理 將較無使用的應用程式切成不允許自動啟動
不好意思，請您幫我點選系統更新   檢查更新
第二張圖幫我在螢幕關機字按住不放
好的請稍候 用適當大小紙箱包裝，用軟填充物填滿固定避免運送時碰撞毀損，並把我們維修單號貼在外部
您可以於PLAY商店 找到華碩通話 這個APP 確認是否還有更新版本
您好，請您至PLAY商店幫我更新華碩相機在嘗試看看謝謝您
了解 這部份可以協助反應建議做3槽的部份
您可以到皇家俱樂部購買變壓器
恢復原廠設定還是卡住無法開機 建議您送檢測 有可能是硬體上有異常
您好 重新開機可離開該模式
因為線上無法確認機檯的實際狀況，建議您將機檯拿回後送到我們的皇家俱樂部，請人員做協助
基本上2.0無法支援到3.0的裝置
如產品部分目前須透過充電組連結使用，可能屬電池周邊硬體模組故障，這部分建議您將產品報修，由維修單位檢測與了解
建議您到PLAY商店搜尋 APP進行更新試試看 音目前有許多APP開發尚未支援ANDROID 7.0 因此確實可能無法使用
您好 Z170C平板無通話功能是正常的
請給我桌機的序號 第三碼為P
幫我們到設定＞＞音效通知＞＞　其他音效＞＞開啟　觸控時震動 測試一下
建議可至附近3C賣場協助確認 或是協助反映至皇家俱樂部請人員協助確認能否訂購
關於您更改地址的部分，也會一併通報
筆電的話繪在機台底部有一

機台幫您確認 最高只支援到4G，沒辦法在往上擴充
請您重裝音效驅動程式排除試試
設定>定位>GOOGLE位置資訊分享
機台在薩爾瓦多這邊並無保固，若要送修建議是到其他國家地區，才會有保固保修的服務
長壓其中一張照片 出現打勾舊可選取多張了
好的 如果還是沒辦法需請您嘗試硬體還原(提醒您資料將遺失)
您好，今日工程師有撥打電話聯絡，但是並沒有撥通。
那代表沒有成功進入工程模式 幫我關機後 試著音量上下一起按 配上電源鍵
這部分目前試還有相關的資訊  真的很報歉 建議您還是持續關注我門官網部分
請您確認網路是否有連線
幫我們長壓該選項 清除網路重新設定一次
產品序號(SN或SSN)購買盒子底部條碼貼紙
了解，如果是這樣的狀況，建議您讓我們把機台收回來維修檢測
基本上原廠比較不建議客戶改灌系統  您可以插上USB重開機連點 ESC會進入開機選單  VM03YK6 選擇USB開機 即可
無法連線出現什麼錯誤訊息
請您先至PLAY商店>搜尋ZENUI撥號器>是否可以更新
這裡僅提供文字服務，語音服務請致電0800-093-456
請您提供E-MAIL
如果還是有相同問題需為您做送修
那可能須請您送修檢測
如您測試過後仍然有收訊問題 幫我們確認韌體是否有更新 設定>>關於>>系統更新
您設定好圖形鎖後回到鎖定螢幕有個選項是顯示解鎖圖形關閉即可
了解，可能必須提供產品型號才有辦法詳細了解原因。
需送修請工程師報健 報價 現在已經沒有線上報價服務 都需收回檢測報價
HTTPS://WWW.ASUS.COM/TW/NOTEBOOKS/X550VX/ 可以參照 X550VX
您要不要重開機試試看
您好 手機不支援HDMI功能
在安全模式下 您可以再確認是否在聲音是正常的
系統一定時間沒有回應會斷線 或是 網路不穩定有可能會斷線
可以送服務點處理 會收500元安裝費 但沒測試過 相容性無法保證
可能還是需要您的維修單號，才有辦法確認，基本上只會針對您的狀況做檢測。
確認目前機台已寄出 煩請留意這幾天的物流
遊戲部分因為目前下架 可能是要安裝APK或是重下載紀錄下載看看
ASUSTEK COMPUTER (AUSTRALIA) PTY LTD
1300-2787-88 (09:00-18:00 MON. ~ FRI.)
BRISBANE
UNIT 3, 544 KESSELS ROAD M

麻煩您長壓面板空白處>>主畫面編輯>>選擇字型
手機完成還原程序後會停留在這個畫面，請選擇 "REBOOT SYSTEM NOW" 並按下電源鍵重新開機即可。
因為這方面，電腦的USB 孔，會視主機板或USB模組的供電有所不同，這方面充電的速率不一定相同，一般也會低於充電組的供電
產品部份產品當中功能，需要將產品解鎖後才能使用
幫您確認您的韌體已經是最新版本
ZENBOOK產品應該是為筆記型電腦產品
您方便先確認在PLAY商店>YOUTUBE>是否可以更新  因為剛剛這邊有測試，YOUTUBE在播放時應是不會進入休眠的
非常抱歉，因UNLOCKTOOL 雖由網站提供，但UNLOCK相關，非產品原廠原生提供一般操作方式或使用功能，於本端部分無法提供相關資訊
如果您是行動數據開啟的狀況 一般8小時 15%  算是省電的
您應該試螢幕會自動亂跳
一般來說產品部分，也可於關機時充電 如時無法充電，建議您比對看看不同充電組的狀況
正常快充的部分是不會影響到的
不好意思，目前安卓6 因為系統的設置規定 ，是無法將 應用程式 移轉 SD 卡 的 。
雙卡部分，目前許多產品僅有支援 4+2卡片功能， 第二張卡會降為2G 但2G服務可能受於後續的限制，電信可能不在提供會建議您參考4+3產品 如ZE520KL /ZE552KL 產品 ZE520KL  32G/3G  建議售價 7,990
目前6.0版本 簡訊APP為 MESSENGER程式
由於客服中心未經手配件銷售 建議年洽詢經銷商或是皇家喔
只能一張SIM卡和一張記憶卡 還有同時兩張SIM卡不能放記憶卡
您可以致電皇家確認，或是這邊幫您反映給工程師
那您就是到 ASUSSOUNDRECORD 錄音資料夾裡面看一下檔案
查詢此機台不支援HDMI功能
您長壓桌面後 點擊桌面設定 在單頁模式選擇為雙頁在使用看看
但提醒您 快地需要請您自行包裝喔
可以 查保固請您提供產品的序號SM/SSN:
這部分需要送修由人員協助更換
此機台是有支援WIN7、WIN10
稍等一下  幫您確認
您可於設定>>顯示>>桌布>>選擇"圖片庫"套用
不建議改灌系統，由於原廠未測試，可能會有相容性問題
一般來說產品右下雙方塊按鍵，點選會出現最近開啟的應用程式屬正常現象
可以請您 前往 小工具內 把 天氣 拖曳就可以
UX501VW 屬於 UX501V  的詳細型

建議您使用CHROME登陸臉書測試看看謝謝您
地點與連絡部分資訊，尚無對外提供，都是由服務端或維修端安排回電 請您稍候幫您查詢
建議可洽皇家 俱樂部
非常抱歉，本端屬於ASUS產品服務系統，HTC產品狀況，建議您洽HTC 服務單位
基本上無法直接設定存到記憶卡哦 可以用複製的方式到記憶卡 謝謝
有一個方式 外接螢幕看看 看有沒有畫面 如果沒畫面 就只能送修
因每個人使用需求不同  建議您可洽詢經銷商 請人員協助
建議您可以將機台背蓋拆下 重插拔電池 但請注意拆機時不要造成損壞會判定為人損
目前無VR眼鏡配件相容性測試
有的產品支援OTG功能
確認中 我確認到 您的筆電 4G，最高可擴充到8G 但是 4G那條要拔掉
您好 目前6.0已不支援APP2SD功能  後續是否有變更都會在官網公告
這部分建議您可以到皇家俱樂部檢測，或是由這邊幫您收會檢測
您螢幕最上端往下滑 有一個自動旋轉幫我按下去 設定 顯示 自動旋轉螢幕 也可以開啟
ZC553KL 是4G  產品 
 
SIM 1: 2G/3G/4G MICRO SIM 卡

SIM 2: 2G/3G/4G NANO SIM 卡
可以移除後重新註冊就好
確認到目前資訊最大是支援到4G 他有一個額外的插槽 但是額外擴充原廠沒測試過 不能保證相容性
不好意思 由於LINE 屬於第三方軟體 建議您洽詢 LINE客服備份對話紀錄的方式
建議您重裝音效驅動排除試試
不好意思，要煩請您重新選擇筆記型電腦由專員為您作詳盡解說;敬請見諒,謝謝
建議您先等他更新完
手機操作相關簡易教學您可參考 ZENCARE  HTTPS://WWW.ASUS.COM/TW/SUPPORT/ZENCARE/
麻煩您幫我看 機台底部 貼紙上的SN
基本上套裝筆電並未針對其他配件做過測試 無法跟您保證相容性 但是就規格上來看 此款變壓器有支援65W 
可以到府，但人員需安排船運時間 故可能會影響用戶使用時間 請見諒
ZE552KL  是5.5吋  ZE520KL 是5.2吋 
有查詢到您的紀錄 比較不好意思 造成您的困擾 這部分建議您報修並將您的狀況做反應維修單位 會請工程師留意您的機台
這個部分比較建議將機台再次送測
因線上客服為客服部，無法得知相關料件報價 都是要送修才可知道
因上方看到 顯示不符合資格 若有您的PLAY商店付款一律 *疑慮 您可以在與GO

這部分鎖定時間是沒有辦法另外設定時間的 產品確認進入待機後，即會需要再次輸入螢幕解鎖的
再正常升級，是不會有無法開機的狀況。
您可以透過PLAY TO功能 對應到您的顯示產品支援MIRACAST進行配對。
活動部分建議撥打4498088洽詢活動小組相關問題
出機 應有還原系統， 可自行還原
建議您使用其他產品測試此WIFI環境是否正常
請您撥打0800093456按1按2再按1
您好，因為您電池本身保固到2016/03/14 ，
您好，因預購會填寫到帳號密碼，因屬個資法保護，無法協助為您訂購
延伸保固主要是針對家用筆電的部份，讓產品保固延長到三年 家用桌機因本身保固已經是三年了，所以不需要啟用
這樣的話，可能還是與您電腦廠牌的客服確認會比較好 不同廠牌在驅動設計上會有所差異
您可檢查網路 或是更新網卡驅動
因為您的BIOS已是最新版 有可能是硬體問題 建議備份後報修
不好意思 您是說 更新後 無法開機 有無試過 長按電源鍵30秒試試看
嘗試看看進入相機>左上齒輪>恢復預設值嘗試
使用 ASUS SMART GESTURE  設定
請提供給我幫您先確認機台的部份
建議將SIM卡拔出 輕輕擦拭金手指重新插入測試
不太懂 可否提供序號
您好  麻煩您到 設定>>勿擾設定 確認是否有開啟 如不要該功能 請選關閉
這部分建議您可以到皇家服務點 購買 您可以先電話諮詢是否有現貨 在購買
前一位客服已協助您反應到宜蘭皇家了,謝謝!
APP如有異常 大多是與更新相關 建議您APP重新安裝 確認是否有更新
目前在使用原廠皮套 正常使用是沒有您說的燙手狀況發生
能否請您到 設定/關於/狀態/序號提供給我一下
建議您可以嘗試看看進入安全模式嘗試看看是否會有這樣的情況產生
一般是單按一次就會有，預覽長度的畫面跳出，如都沒有顯示出，建議您可以先透過PLAY 商店更新華碩相機 並更新手機韌體版本再確認看看
建議您可以到華碩皇家俱樂部購買
目前線上尚無升級資訊，建議您留意官網公告為主喔
因為安全模式下如是顯示正常 就有可能是第三方軟體產生的 所以要請您確認第三方程式
非常抱歉，受於各通路或販售據點價格皆不同，此方面較無法值皆判斷 
這方面建議您可以參考ASUS網站資訊
HTTPS://WWW.ASUS.COM/TW/PHONE/
您先稍等 我這邊調閱下記錄
若是完修還有異狀的話 這邊可以幫您反映

In [40]:
tmp = jieba.cut("HTTPS://WWW.ASUS.COM/ZENTALK/TW/THREAD-168834-1-1.HTML", cut_all = False)
print(list(tmp))

['HTTPS', ':', '/', '/', 'WWW', '.', 'ASUS', '.', 'COM', '/', 'ZENTALK', '/', 'TW', '/', 'THREAD', '-', '168834', '-', '1', '-', '1', '.', 'HTML']


In [41]:
'''count = 0
for i in range(len(training_tup_list)):
    if("HTTP" in training_tup_list[i][1]):
        #count+=1
        for j in range(len(training_tup_list[i][1])):
            if training_tup_list[i][1][j] == 'H' and training_tup_list[i][1][j+1] =="T"and training_tup_list[i][1][j+2] =="T":
                loc_start = j
                break
        
        for j in range(loc_start, len(training_tup_list[i][1])):
            loc_end = j
            if training_tup_list[i][1][j].isalpha() or training_tup_list[i][1][j].isdigit() or training_tup_list[i][1][j] in '?=+_/:.-%':
                 continue
            else:
                break
        print(training_tup_list[i][1][loc_start:loc_end])
        if len(training_tup_list[i][1][loc_start:loc_end]) < 10:
            count+=1
            print("===="+str(loc_start) +"===" +str(loc_end)+"====")
            print(training_tup_list[i][1])
            print("===")
            
print(count)'''
            
            
                    
    

'count = 0\nfor i in range(len(training_tup_list)):\n    if("HTTP" in training_tup_list[i][1]):\n        #count+=1\n        for j in range(len(training_tup_list[i][1])):\n            if training_tup_list[i][1][j] == \'H\' and training_tup_list[i][1][j+1] =="T"and training_tup_list[i][1][j+2] =="T":\n                loc_start = j\n                break\n        \n        for j in range(loc_start, len(training_tup_list[i][1])):\n            loc_end = j\n            if training_tup_list[i][1][j].isalpha() or training_tup_list[i][1][j].isdigit() or training_tup_list[i][1][j] in \'?=+_/:.-%\':\n                 continue\n            else:\n                break\n        print(training_tup_list[i][1][loc_start:loc_end])\n        if len(training_tup_list[i][1][loc_start:loc_end]) < 10:\n            count+=1\n            print("===="+str(loc_start) +"===" +str(loc_end)+"====")\n            print(training_tup_list[i][1])\n            print("===")\n            \nprint(count)'

In [42]:
with open("ans_classification_set_10.txt","w", encoding='utf-8') as f:
    for i in range(len(ans_relevant_set)):
        f.write("Answer group %d : length is %d\n"% (i, len(ans_relevant_set[i])))
        count = 0
        for j in ans_relevant_set[i]:
            
            f.write("%d:          %s\n"% (count,training_tup_list[j][1]))
            count+=1
        
        f.write("\n\n\n")
        f.write("=======================================================\n\n\n")
        
    

In [43]:
district_set = []  #尚未加入分類  為了處裡筆電顯卡特殊問題
for i in range(len(training_tup_list)):
    if "目前即時通服務尚未提供" in training_tup_list[i][1]:
        local_start = 0
        local_end = 0
        for num in range(len(training_tup_list[i][1])):
            if training_tup_list[i][1][num] == "供":
                loc_start = num
            elif training_tup_list[i][1][num] == '專' or training_tup_list[i][1][num] == '相':
                loc_end = num
                break
        
        
        district_set.append((training_tup_list[i][1][loc_start+1:loc_end],i))
dis_index_set = []
for i in district_set:
    dis_index_set.append(i[1])
    
dis_ans_set =[] 
for i in range(len(district_set)):
    if district_set[i][1] in dis_index_set:
        tmp_list = [district_set[i]]
        for j in range(i+1, len(district_set)):
            if district_set[i][0] == district_set[j][0] and district_set[j][1] in dis_index_set:
                tmp_list.append(district_set[j])
                dis_index_set.remove(district_set[j][1])
        if len(tmp_list) > 1:
            dis_ans_set.append(tmp_list)
        
for i in dis_ans_set:
    print(i)   

[('零售主機板/顯示卡', 45), ('零售主機板/顯示卡', 47), ('零售主機板/顯示卡', 103), ('零售主機板/顯示卡', 128), ('零售主機板/顯示卡', 400), ('零售主機板/顯示卡', 403), ('零售主機板/顯示卡', 432), ('零售主機板/顯示卡', 534), ('零售主機板/顯示卡', 591), ('零售主機板/顯示卡', 804), ('零售主機板/顯示卡', 818), ('零售主機板/顯示卡', 831), ('零售主機板/顯示卡', 843), ('零售主機板/顯示卡', 861), ('零售主機板/顯示卡', 872), ('零售主機板/顯示卡', 916), ('零售主機板/顯示卡', 964), ('零售主機板/顯示卡', 988), ('零售主機板/顯示卡', 1028), ('零售主機板/顯示卡', 1036), ('零售主機板/顯示卡', 1058), ('零售主機板/顯示卡', 1090), ('零售主機板/顯示卡', 1094), ('零售主機板/顯示卡', 1097), ('零售主機板/顯示卡', 1153), ('零售主機板/顯示卡', 1155), ('零售主機板/顯示卡', 1214), ('零售主機板/顯示卡', 1255), ('零售主機板/顯示卡', 1256), ('零售主機板/顯示卡', 1309), ('零售主機板/顯示卡', 1333), ('零售主機板/顯示卡', 1407), ('零售主機板/顯示卡', 1483), ('零售主機板/顯示卡', 1657), ('零售主機板/顯示卡', 1718), ('零售主機板/顯示卡', 1800), ('零售主機板/顯示卡', 1854), ('零售主機板/顯示卡', 2185), ('零售主機板/顯示卡', 2316), ('零售主機板/顯示卡', 2400), ('零售主機板/顯示卡', 2440), ('零售主機板/顯示卡', 2588), ('零售主機板/顯示卡', 2632), ('零售主機板/顯示卡', 2675), ('零售主機板/顯示卡', 2703), ('零售主機板/顯示卡', 2769), ('零售主機板/顯示卡', 2771), ('零售主機板/顯示卡', 2816), ('零售主機板/顯示卡

In [44]:
training_tup_list[5966][1]

'先向您說聲抱歉,目前即時通服務尚未提供贈品相關服務,建議您直接撥打 4498088  由贈品部門為您作詳盡解說;此區目前為手機/平版專區,敬請見諒,謝謝  服務時間為週一~週五09:00~18:00   '

In [45]:
tmp_Wrong_ans_int_list = []
for i in range(len(training_tup_list)):#隨機找一句不是這句答案的答案
    tmp_int_set = randint(0, len(ans_relevant_set)- 1)
    while( i  in ans_relevant_set[tmp_int_set]):
        tmp_int_set = randint(0, len(ans_relevant_set)- 1)
    
    tmp_int_ans = randint(0,len(ans_relevant_set[tmp_int_set]) - 1)
    wrong_ans_num = ans_relevant_set[tmp_int_set][tmp_int_ans]
    
    tmp_Wrong_ans_int_list.append(wrong_ans_num)

#print(tmp_Wrong_ans_int_list)
test1_df = pd.DataFrame(np.array(tmp_Wrong_ans_int_list))
test1_df.to_pickle("Wrong_ans_int_set.pkl")

In [46]:
tmp_Wrong_ans_int_list[1]

4188

In [47]:
print(training_tup_list[1][1])
print(training_tup_list[2721][1])

您好 若於料件更換 需將機台送修 由工程人員來做進一步檢測及後續報價。
不好意思，請您稍候片刻，我會立即為您查詢，謝謝。 不好意思 幫您卻認到 目前B9440UA 僅支援WIN10的系統 不建議您灌WIN7


In [48]:
val_ans_word_list = []
for i in range(len(validation_tup_list)):#把每個ANS裡面如果有出現詞那個詞就設成1
    
    list2 = jieba.lcut(validation_tup_list[i][1], cut_all = False)
    tmp = np.zeros((len(vec_model.wv.vocab), 1))#每一個詞都是一個所有詞的數量的ONEHOT
    for word in list2:
        if word in vec_model.wv.vocab:
            tmp[word_index[word]] = 1
    val_ans_word_list.append(tmp)

In [49]:
candidate_100_val = []

val_ans_class = []
# i: Every records
for i in range(len(validation_tup_list)):
    tmp_ans_list = []
    
    ans_grade_set = []
    # j: Every cluster
    for j in range(len(ans_relevant_set)):
        grade_sum = 0
        # k: Every sentence's index in cluster
        for k in range(len(ans_relevant_set[j])):#計算每一群裡面每一句的分數
            tmp_int = ans_relevant_set[j][k]
            grade = np.sum(val_ans_word_list[i] * ans_word_list[tmp_int][0])
            grade_sum += grade
        grade_avg = grade_sum / len(ans_relevant_set[j])#取平均當作那一群對上這一個答案的分數
        ans_grade_set.append((j, grade_avg))
    
    ans_grade_set.sort(key=lambda tup: tup[1], reverse=True)#由大排到小
    val_ans_class.append((ans_grade_set[0], ans_grade_set[1], ans_grade_set[2]))
    for j in range(3, 102):#找第三個到102個  因為前面是正確的
        ans_set_num = ans_grade_set[j][0]
        ans_num = randint(0, len(ans_relevant_set[ans_set_num])-1)
        tmp_int = ans_relevant_set[ans_set_num][ans_num]
        tmp_ans_list.append(training_tup_list[tmp_int][1])
        
    
    tmp_ans_list.append(validation_tup_list[i][1])
    
    candidate_100_val.append(tmp_ans_list)
    
    if i % 50 == 0:#算時間
        print(i)       

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900


In [50]:
test_df = pd.DataFrame.from_records(candidate_100_val)
test_df.to_pickle("candidate_100_ans.pkl")

In [51]:
"""
count = 0#看validation的答案的分類狀況
for i in val_ans_class:
    
    print(validation_tup_list[count][1] + '\n')
    for tup in i:
        tmp_int = ans_relevant_set[tup[0]][0]
        print("%s         %f\n" %(training_tup_list[tmp_int][1], tup[1]))
    count += 1
    print("===============")
"""    

'\ncount = 0#看validation的答案的分類狀況\nfor i in val_ans_class:\n    \n    print(validation_tup_list[count][1] + \'\n\')\n    for tup in i:\n        tmp_int = ans_relevant_set[tup[0]][0]\n        print("%s         %f\n" %(training_tup_list[tmp_int][1], tup[1]))\n    count += 1\n    print("===============")\n'